In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## 함수 목록

In [34]:
# 두 좌표 사이 거리 측정 함수
def haversine(lat1, lon1, lat2, lon2):
    """
    두 좌표 사이의 실제 지구 표면 거리(km)를 계산.
    단순 유클리드 거리 대신 구면 기하학 공식을 사용해
    위도가 높아질수록 경도 간격이 좁아지는 왜곡을 보정함.
    """
    R = 6371  # 지구 반경 (km)
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    return R * 2 * np.arcsin(np.sqrt(a))

# 좌표의 인접 초등학교 찾는 함수
def find_nearest_school(point_df, safezone_df):
    """
    df 전체와 거리를 한 번에 벡터 연산으로 계산 후
    거리가 최소인 행의 시설명을 반환.
    루프 없이 numpy 배열 연산을 쓰므로 데이터가 많아도 빠름.
    """
    distances = haversine(
        point_df['위도'].values, point_df['경도'].values,
        safezone_df['위도'].values,
        safezone_df['경도'].values
    )
    nearest_idx = np.argmin(distances)
    return safezone_df.iloc[nearest_idx]['시설명']

In [33]:
from sklearn.neighbors import BallTree
import numpy as np

def find_nearest(point_df, safezone_df, k=1):
    # BallTree 생성 (haversine은 라디안 단위 필요)
    tree = BallTree(
        np.radians(safezone_df[['위도', '경도']].values),
        metric='haversine'
    )
    
    # 쿼리: 각 포인트마다 가장 가까운 1개 찾기
    _, nearest_idx = tree.query(
        np.radians(point_df[['위도', '경도']].values),
        k=k)
    
    return safezone_df.iloc[nearest_idx.flatten()][['시설종류', '대상시설명']].values

In [ ]:
# 위도 경도 매핑
import requests
import time
from dotenv import load_dotenv
import os

load_dotenv()  
KAKAO_API_KEY = os.getenv("KAKAO_API_KEY")

def get_coordinates(address):
    """
    카카오 지오코딩 API에 주소를 보내 위도/경도를 반환하는 함수
    실패 시 (None, None) 반환
    """
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        result = response.json()
        
        if result["documents"]:
            lat = float(result["documents"][0]["y"])  # 위도
            lon = float(result["documents"][0]["x"])  # 경도
            return lat, lon
        else:
            return None, None
    except Exception as e:
        print(f"오류 발생: {address} -> {e}")
        return None, None

# 주소 컬럼명 확인 후 수정 (이미지 기준: "소재지" 또는 "주소")
address_col = "주소"  # 또는 "주소"

# # 위도/경도 컬럼 추가
# latitudes = []
# longitudes = []

# for addr in police_location_df[address_col]:
#     lat, lon = get_coordinates(addr)
#     latitudes.append(lat)
#     longitudes.append(lon)
#     time.sleep(0.3)  # API 호출 제한 방지 (초당 최대 10회)

# police_location_df["위도"] = latitudes
# police_location_df["경도"] = longitudes

# # 결과 확인
# police_location_df.head()

## 1. 초등학교현황 -> 어린이 보호구역 현황 데이터 사용
https://data.gg.go.kr/portal/data/service/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=0UH7S75R45EM2FTI971A20528353&infSeq=1&order=&loc=

In [14]:
school_df = pd.read_csv('./raw_data/초등학교현황.csv', encoding='cp949')
school_df

,설립구분명,시설명,전화번호,소재지지번주소,소재지도로명주소,소재지우편번호,WGS84위도,WGS84경도
0,공립,가림초등학교,02-803-3329,경기도 광명시 하안동 297번지,경기도 광명시 금당로 11-7,14305.0,37.458404,126.878378
1,공립,가평마장초등학교,031-582-2756,경기도 가평군 가평읍 마장리 357번지,경기도 가평군 가평읍 각담말길 15,12409.0,37.860144,127.514733
2,공립,가평초등학교,031-582-2491,경기도 가평군 가평읍 읍내리 553번지,경기도 가평군 가평읍 향교로 23,12417.0,37.831231,127.507739
3,공립,대성초등학교,031-584-0621,경기도 가평군 청평면 대성리 399-16번지,경기도 가평군 청평면 경춘로 75,12457.0,37.683125,127.377707
4,공립,목동초등학교,031-581-0615,경기도 가평군 북면 이곡리 13-45번지,경기도 가평군 북면 석장모루길 13,12407.0,37.879306,127.548123
...,...,...,...,...,...,...,...,...
1398,공립,다솜초등학교,031-932-8567,경기도 고양시 일산동구 풍동 1232번지,경기도 고양시 일산동구 숲속마을로 139,10303.0,37.672654,126.796583
1399,공립,신지초등학교,031-836-8123,경기도 양주시 백석읍 복지리 156번지,경기도 양주시 백석읍 양주산성로574번길 20,11510.0,37.788262,126.990184
1400,공립,다산한강초등학교,031-523-7500,경기도 남양주시 다산동 6236번지,경기도 남양주시 다산지금로146번길 11,12284.0,37.599431,127.172380
1401,공립,남양주다산초등학교,031-522-3200,경기도 남양주시 다산동 6049번지,경기도 남양주시 다산중앙로146번길 55,12285.0,37.626685,127.158570


In [15]:
school_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1403 entries, 0 to 1402
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   설립구분명     1403 non-null   object 
 1   시설명       1403 non-null   object 
 2   전화번호      1403 non-null   object 
 3   소재지지번주소   1398 non-null   object 
 4   소재지도로명주소  1381 non-null   object 
 5   소재지우편번호   1385 non-null   float64
 6   WGS84위도   1382 non-null   float64
 7   WGS84경도   1382 non-null   float64
dtypes: float64(3), object(5)
memory usage: 87.8+ KB


In [16]:
school_df[school_df.isna().values.any(axis=1)]

,설립구분명,시설명,전화번호,소재지지번주소,소재지도로명주소,소재지우편번호,WGS84위도,WGS84경도
20,공립,백마초등학교장항분교장,031-301-2004,NaN,NaN,NaN,NaN,NaN
56,공립,지축초등학교,02-381-5166,NaN,NaN,NaN,NaN,NaN
214,공립,부천대명초등학교,070-7099-8504,경기도 부천시 오정구 오정동 127-8번지 부천대명초등학교,NaN,14426.0,37.527705,126.797177
383,공립,소래초등학교,031-311-9050,경기도 시흥시 호현로 27번길,NaN,NaN,NaN,NaN
518,공립,보라초등학교,031-282-2770,경기도 용인시 기흥구 금화로 105,NaN,17073.0,NaN,NaN
545,공립,지평초등학교일신분교장,0193321731,NaN,NaN,NaN,NaN,NaN
551,공립,점동초등학교뇌곡분교장,031-882-7549,NaN,NaN,NaN,NaN,NaN
570,공립,원삼초등학교두창분교장,031-322-8014,NaN,NaN,NaN,NaN,NaN
586,공립,발곡초등학교,031-878-8372,경기도 의정부시 동일로454번길,NaN,NaN,NaN,NaN
587,공립,배영초등학교,031-845-5402,경기도 의정부시 가능로 135,NaN,11686.0,NaN,NaN


In [17]:
# 일단 결측치가 있는 행은 제거. 결측치가 있는 행들이 성남 데이터가 아니므로 제거해도 무방할 것으로 판단됨
school_df = school_df.dropna()
school_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1381 entries, 0 to 1402
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   설립구분명     1381 non-null   object 
 1   시설명       1381 non-null   object 
 2   전화번호      1381 non-null   object 
 3   소재지지번주소   1381 non-null   object 
 4   소재지도로명주소  1381 non-null   object 
 5   소재지우편번호   1381 non-null   float64
 6   WGS84위도   1381 non-null   float64
 7   WGS84경도   1381 non-null   float64
dtypes: float64(3), object(5)
memory usage: 97.1+ KB


## 1-1. 어린이 보호구역 현황 -> 메인으로 사용

In [36]:
safezone_df = pd.read_csv('./raw_data/어린이보호구역현황_(성남시).csv')
safezone_df.head()

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도,관리기관명,관할경찰서명,CCTV설치여부,CCTV설치대수,보호구역도로폭,데이터기준일자
0,유치원,배성유치원,경기도 성남시 분당구 판교로 624-1,경기도 성남시 분당구 야탑동 397,37.405999,127.142401,경기도 성남시청,분당경찰서,Y,1,NaN,2025-11-11
1,유치원,이솔유치원,경기도 성남시 분당구 미금로 232,경기도 성남시 분당구 금곡동 133-1,37.352756,127.112602,경기도 성남시청,분당경찰서,Y,5,NaN,2025-11-11
2,유치원,성모 유치원,경기도 성남시 분당구 구미로 130번길 20,경기도 성남시 분당구 구미동 238,37.340099,127.124419,경기도 성남시청,분당경찰서,Y,4,NaN,2025-11-11
3,유치원,판교샘유치원,경기도 성남시 분당구 산운로 98,경기도 성남시 분당구 운중동 956,37.392021,127.072039,경기도 성남시청,분당경찰서,Y,1,NaN,2025-11-11
4,유치원,건영장안유치원,경기도 성남시 분당구 장안로 25번길 28,경기도 성남시 분당구 분당동 67,37.371127,127.140849,경기도 성남시청,분당경찰서,Y,1,NaN,2025-11-11


In [37]:
# 시설 종류, 대상시설명, 도로명주소, 지번주소, 위도, 경도만 추출
safezone_df = safezone_df[['시설종류', '대상시설명', '소재지도로명주소', '소재지지번주소', '위도', '경도']]
safezone_df.head()

,시설종류,대상시설명,소재지도로명주소,소재지지번주소,위도,경도
0,유치원,배성유치원,경기도 성남시 분당구 판교로 624-1,경기도 성남시 분당구 야탑동 397,37.405999,127.142401
1,유치원,이솔유치원,경기도 성남시 분당구 미금로 232,경기도 성남시 분당구 금곡동 133-1,37.352756,127.112602
2,유치원,성모 유치원,경기도 성남시 분당구 구미로 130번길 20,경기도 성남시 분당구 구미동 238,37.340099,127.124419
3,유치원,판교샘유치원,경기도 성남시 분당구 산운로 98,경기도 성남시 분당구 운중동 956,37.392021,127.072039
4,유치원,건영장안유치원,경기도 성남시 분당구 장안로 25번길 28,경기도 성남시 분당구 분당동 67,37.371127,127.140849


In [38]:
safezone_df.columns

Index(['시설종류', '대상시설명', '소재지도로명주소', '소재지지번주소', '위도', '경도'], dtype='object')

In [39]:
safezone_raw =safezone_df.copy()
safezone_df = safezone_df.astype({'위도': 'float64', '경도': 'float64'}, errors='ignore')
safezone_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시설종류      142 non-null    object 
 1   대상시설명     142 non-null    object 
 2   소재지도로명주소  142 non-null    object 
 3   소재지지번주소   142 non-null    object 
 4   위도        142 non-null    float64
 5   경도        142 non-null    float64
dtypes: float64(2), object(4)
memory usage: 6.8+ KB


In [232]:
safezone_df.to_csv('../../Preprocessing/외부안전위험요소/어린이보호구역_위치정보.csv', index=False)

## 3. 경기도아동안전지킴이집현황.csv
https://data.gg.go.kr/portal/data/service/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=A484FNEWKSWX3IXUSJ1L29703862&infSeq=1&order=&loc=&downloadType=&SAFE_FACLT_NM=&REFINE_ROADNM_ADDR=&REFINE_LOTNO_ADDR=

- 수집기간 : 2019 - 2026-02-23           

In [20]:
child_shelter_df = pd.read_csv('./raw_data/경기도아동안전지킴이집현황.csv', encoding='cp949')
child_shelter_df.head()

,안전시설명,안전시설전화번호,소재지도로명주소,소재지지번주소,위도,경도,현존주소여부
0,우리자동차운전학원,031-582-0769,경기도 가평군 가평읍 태봉두밀로 34-71,경기도 가평군 가평읍 상색리 171-38번지,37.804569,127.486789,S
1,경희대튼튼체육관,031-582-6943,경기도 가평군 가평읍 석봉로 146,경기도 가평군 가평읍 읍내리 329-21번지,37.828148,127.511625,S
2,CU가평군청점,NaN,경기도 가평군 가평읍 석봉로 175,경기도 가평군 가평읍 읍내리 495-30번지,37.830591,127.510341,S
3,GS25가평군청점,031-582-5175,경기도 가평군 가평읍 석봉로 163,경기도 가평군 가평읍 읍내리 495-32번지,37.829716,127.510636,S
4,다래향,031-582-5552,경기도 가평군 가평읍 향교로 4,경기도 가평군 가평읍 읍내리 535번지,37.829653,127.510295,S


In [21]:
child_shelter_df

,안전시설명,안전시설전화번호,소재지도로명주소,소재지지번주소,위도,경도,현존주소여부
0,우리자동차운전학원,031-582-0769,경기도 가평군 가평읍 태봉두밀로 34-71,경기도 가평군 가평읍 상색리 171-38번지,37.804569,127.486789,S
1,경희대튼튼체육관,031-582-6943,경기도 가평군 가평읍 석봉로 146,경기도 가평군 가평읍 읍내리 329-21번지,37.828148,127.511625,S
2,CU가평군청점,NaN,경기도 가평군 가평읍 석봉로 175,경기도 가평군 가평읍 읍내리 495-30번지,37.830591,127.510341,S
3,GS25가평군청점,031-582-5175,경기도 가평군 가평읍 석봉로 163,경기도 가평군 가평읍 읍내리 495-32번지,37.829716,127.510636,S
4,다래향,031-582-5552,경기도 가평군 가평읍 향교로 4,경기도 가평군 가평읍 읍내리 535번지,37.829653,127.510295,S
...,...,...,...,...,...,...,...
2141,세븐일레븐 봉담신창점,NaN,NaN,경기도 화성시 효행구 봉담읍 수영리 672번지 한울마을신창비바패밀리아파트,37.234563,126.956689,S
2142,(화성서부) 봉담지역아동센타,031-226-8291,경기도 화성시 효행구 봉담읍 북촌길 12-5,경기도 화성시 효행구 봉담읍 와우리 165-13번지,37.218523,126.975739,S
2143,(화성서부) 경희대 새한체육관,031-223-9396,경기도 화성시 효행구 봉담읍 와우로 20,경기도 화성시 효행구 봉담읍 와우리 220-75번지 미래프라자 502호,37.215248,126.969022,S
2144,(화성서부) 명지대 안양체육관,031-298-1443,경기도 화성시 효행구 봉담읍 유리마을길 79,경기도 화성시 효행구 봉담읍 유리 100-1번지 기산아파트 상가 3층,37.175881,126.933408,S


In [22]:
child_shelter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2146 entries, 0 to 2145
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   안전시설명     2146 non-null   object 
 1   안전시설전화번호  1740 non-null   object 
 2   소재지도로명주소  1826 non-null   object 
 3   소재지지번주소   2146 non-null   object 
 4   위도        2076 non-null   float64
 5   경도        2076 non-null   float64
 6   현존주소여부    2146 non-null   object 
dtypes: float64(2), object(5)
memory usage: 117.5+ KB


In [29]:
# 성남시 안전지킴이집만 필터링
child_shelter_sn = child_shelter_df[child_shelter_df['소재지지번주소'].str.contains('성남', na=False)]
child_shelter_sn.head()
child_shelter_sn['현존주소여부'].unique()

array(['S'], dtype=object)

In [ ]:
child_shelter_sn.info()  # 성남시 위도, 경도, 현존주소여부 모두 존재

<class 'pandas.core.frame.DataFrame'>
Index: 152 entries, 751 to 902
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   안전시설명     152 non-null    object 
 1   안전시설전화번호  134 non-null    object 
 2   소재지도로명주소  121 non-null    object 
 3   소재지지번주소   152 non-null    object 
 4   위도        152 non-null    float64
 5   경도        152 non-null    float64
 6   현존주소여부    152 non-null    object 
dtypes: float64(2), object(5)
memory usage: 9.5+ KB


In [45]:
# 하남시 안전지킴이집만 필터링
child_shelter_hn = child_shelter_df[child_shelter_df['소재지지번주소'].str.contains('하남', na=False)]
child_shelter_hn.head()

,안전시설명,안전시설전화번호,소재지도로명주소,소재지지번주소,위도,경도,현존주소여부
1999,cu편의점 감일포웰시티점,NaN,경기도 하남시 감일백제로 70-1,경기도 하남시 감이동 491번지,37.503466,127.163635,S
2000,cu편의점 감일라포레점,NaN,NaN,경기도 하남시 감이동 500번지 포웰시티푸르지오라포레,37.498045,127.165503,S
2001,포웰공인중개사,02-400-1566,경기도 하남시 감일백제로 20,경기도 하남시 감이동 500번지 포웰시티푸르지오라포레 101호,37.498045,127.165503,S
2002,하우스홀드,02-474-7774,경기도 하남시 감북로 36,경기도 하남시 감일동 14-3번지,37.515690,127.161113,S
2003,GS편의점 하남서부점,031-479-4479,경기도 하남시 감북로 56,경기도 하남시 감일동 2-5번지,37.515895,127.163443,S


In [46]:
# 전화번호, 현존주소여부는 일단 제거 
child_shelter_sn.drop(columns=['안전시설전화번호', '현존주소여부'], inplace=True)
child_shelter_hn.drop(columns=['안전시설전화번호', '현존주소여부'], inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_8868\1140227879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_shelter_sn.drop(columns=['안전시설전화번호', '현존주소여부'], inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_8868\1140227879.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_shelter_hn.drop(columns=['안전시설전화번호', '현존주소여부'], inplace=True)


In [47]:
# 인접한 구역 매핑
result = find_nearest(child_shelter_sn, safezone_df)
child_shelter_sn[['시설종류', '대상시설명']] = result  # 시설종류와 대상시설명 컬럼 추가
child_shelter_sn.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_8868\1373772828.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  child_shelter_sn[['시설종류', '대상시설명']] = result  # 시설종류와 대상시설명 컬럼 추가


,안전시설명,소재지도로명주소,소재지지번주소,위도,경도,시설종류,대상시설명
751,(분당)하나약국,경기도 성남시 분당구 미금일로 71,경기도 성남시 분당구 구미동 144-4번지,37.345621,127.111818,어린이집,구미동어린이집
752,(분당)GS25 오리역점,경기도 성남시 분당구 성남대로43번길 10,경기도 성남시 분당구 구미동 158번지,37.340028,127.107817,어린이집,구미동어린이집
753,(분당)세븐일레븐 오리역점,경기도 성남시 분당구 성남대로 38,경기도 성남시 분당구 구미동 185-1번지,37.339312,127.109390,어린이집,구미동어린이집
754,(분당)세븐일레븐 분당성우점,경기도 성남시 분당구 구미로 16,경기도 성남시 분당구 구미동 189-1번지 성우스타우스오피스텔 1층 103호,37.337713,127.110990,초등학교,구미초등학교
755,(분당)기린약국,경기도 성남시 분당구 미금로 48,경기도 성남시 분당구 구미동 205-1번지,37.337713,127.116544,초등학교,구미초등학교


In [48]:
# 저장
child_shelter_sn.to_csv('../../Preprocessing/외부안전위험요소/성남시_아동안전지킴이집현황_전처리.csv')
child_shelter_hn.to_csv('../../Preprocessing/외부안전위험요소/하남시_아동안전지킴이집현황_전처리.csv', index=False)

### 3. 처리 목록
- 현존하지 않는 데이터 제거 
- 전화번호 컬럼 제거 

## 5. 경기도 성남시_관공서 및 주요기관 정보_20251126.csv 
https://www.data.go.kr/data/15032489/fileData.do
https://www.data.go.kr/data/15077036/fileData.do

- 수집 일자: ~ 2025-11-26

In [50]:
police_station_df = pd.read_csv('./raw_data/경기도 성남시_관공서 및 주요기관 정보_20251126.csv')
police_station_df.head()

,구 분,기관명,소 재 지,관할구역,전화번호,데이터기준일자
0,분당구,성남교육지원청,경기도 성남시 분당구 양현로 20,성남시,031-780-2500,2025-11-26
1,수정구,성남세무서,경기도 성남시 수정구 희망로 480,"성남시 수정구, 중원구",031-730-6200,2025-11-26
2,분당구,분당세무서,경기도 성남시 분당구 분당로 23,성남시 분당구,031-219-9200,2025-11-26
3,법원(지원),수원지방법원성남지원,경기도 성남시 수정구 산성대로 451,성남시,031-737-1114,2025-11-26
4,등기소,성남지원등기소,경기도 성남시 수정구 산성대로 451,성남시,1544-0773,2025-11-26


In [51]:
police_station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구 분      54 non-null     object
 1   기관명      54 non-null     object
 2   소 재 지    54 non-null     object
 3   관할구역     54 non-null     object
 4   전화번호     54 non-null     object
 5   데이터기준일자  54 non-null     object
dtypes: object(6)
memory usage: 2.7+ KB


In [52]:
keywords = ['파출소', '지구대', '치안센터', '경찰서']
pattern = '|'.join(keywords)  
police_station_df = police_station_df[police_station_df['기관명'].str.contains(pattern, na=False)]
police_station_df.head()

,구 분,기관명,소 재 지,관할구역,전화번호,데이터기준일자
7,수정구,성남수정경찰서,경기도 성남시 성남대로 1259,수정구,031-182,2025-11-26
8,수정구,수진지구대,경기도 성남시 수정구 모란로 1,"태평1동, 수진2동",031-750-4020,2025-11-26
9,수정구,신흥지구대,경기도 성남시 수정구 시민로 150,"신흥1동, 신흥2동, 신흥3동",031-750-4030,2025-11-26
10,수정구,고등파출소,경기도 성남시 수정구 고등공원로 47,"신촌동, 오야동 등 9개동",031-750-4050,2025-11-26
11,수정구,복정파출소,경기도 성남시 수정구 복정로 36,복정동,031-750-4060,2025-11-26


In [53]:
police_station_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 7 to 34
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구 분      28 non-null     object
 1   기관명      28 non-null     object
 2   소 재 지    28 non-null     object
 3   관할구역     28 non-null     object
 4   전화번호     28 non-null     object
 5   데이터기준일자  28 non-null     object
dtypes: object(6)
memory usage: 1.5+ KB


In [54]:
# 전화번호 및 데이터기준일자 삭제
police_station_df.drop(columns=['전화번호', '데이터기준일자'], inplace=True)

In [ ]:
# 위치 매핑


#### 화성시 파출소 데이터 -> 삭제

In [161]:
police_station_df2 = pd.read_csv('./raw_data/경찰청_전국 지구대 파출소 주소 현황_20251231.csv', encoding='cp949')
police_station_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2047 entries, 0 to 2046
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연번      2047 non-null   int64 
 1   시도청     2047 non-null   object
 2   경찰서     2047 non-null   object
 3   관서명     2047 non-null   object
 4   구분      2047 non-null   object
 5   주소      2047 non-null   object
dtypes: int64(1), object(5)
memory usage: 96.1+ KB


In [162]:
police_station_df2['시도청'].unique()

array(['서울청', '부산청', '대구청', '인천청', '광주청', '대전청', '울산청', '세종청', '경기남부청',
       '경기북부청', '강원청', '충북청', '충남청', '전북청', '전남청', '경북청', '경남청', '제주청'],
      dtype=object)

In [163]:
mask = (
    (police_station_df2['시도청'].str.contains('경기남부청|경기북부청', na=False)) &
    (police_station_df2['주소'].str.contains('화성', na=False))
)

result = police_station_df2[mask]
result.count()

연번     17
시도청    17
경찰서    17
관서명    17
구분     17
주소     17
dtype: int64

In [173]:
police_station_df2 = result
police_station_df2

,연번,시도청,경찰서,관서명,구분,주소
730,731,경기남부청,화성서부,남양,파출소,화성시 남양시장로 47
731,732,경기남부청,화성서부,발안,지구대,화성시 3.1만세로 1095
733,734,경기남부청,화성서부,매송,파출소,화성시 매송면 화성로 2249
734,735,경기남부청,화성서부,송산,파출소,화성시 송산포도로 98
735,736,경기남부청,화성서부,서신,파출소,화성시 서신면 매화1길 31
736,737,경기남부청,화성서부,우정,파출소,화성시 우정읍 조암남로 31
737,738,경기남부청,화성서부,팔탄,파출소,화성시 팔탄면 서촌길 9
738,739,경기남부청,화성서부,비봉,파출소,화성시 비봉면 양노로 56
739,740,경기남부청,화성서부,마도,파출소,화성시 마도면 석교로 179
740,741,경기남부청,화성서부,양감,파출소,화성시 양감면 은행나무로 263


In [165]:
police_station_df.columns

Index(['구 분', '기관명', '소 재 지', '관할구역', '전화번호', '데이터기준일자'], dtype='object')

In [178]:
df1 = police_station_df[['구 분', '기관명', '소 재 지']].rename(columns={
    '소 재 지': '주소'
})

# df2: 구분 값 변환 (화성동탄 → 동탄구, 화성서부 → 그대로)
df2 = police_station_df2.copy()

df2['구 분'] = df2['경찰서'].map({
    '화성동탄': '동탄구',
    '화성서부': '화성서부'
})

# df2: 기관명 생성 (관서명 + 구분 합치기 → 예: '남양파출소')
df2['기관명'] = df2['관서명'] + df2['구분']

# df2: df1에 맞는 컬럼만 유지
df2 = df2[['구 분', '기관명', '주소']]

# 병합
df_merged = pd.merge(
    df1, df2,
    on=['구 분', '기관명', '주소'],
    how='outer'
)

print(df_merged.shape)
df_merged.head()  # 28+17 =45개

(45, 3)


,구 분,기관명,주소
0,동탄구,동탄2지구대,경기도 화성시 동탄영천로 70
1,동탄구,동탄3지구대,경기도 화성시 왕배산길 6
2,동탄구,동탄지구대,경기도 화성시 노작로 226-1
3,동탄구,반월파출소,경기도 화성시 영통로 27번길 25-19
4,동탄구,안용파출소,경기도 화성시 용주로 32번길 4


In [180]:
police_location_df = df_merged.copy()

위도/경도 매핑

In [121]:
# %pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [122]:
import requests
import time
from dotenv import load_dotenv
import os

load_dotenv()  
KAKAO_API_KEY = os.getenv("KAKAO_API_KEY")

def get_coordinates(address):
    """
    카카오 지오코딩 API에 주소를 보내 위도/경도를 반환하는 함수
    실패 시 (None, None) 반환
    """
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        result = response.json()
        
        if result["documents"]:
            lat = float(result["documents"][0]["y"])  # 위도
            lon = float(result["documents"][0]["x"])  # 경도
            return lat, lon
        else:
            return None, None
    except Exception as e:
        print(f"오류 발생: {address} -> {e}")
        return None, None

In [181]:
# 주소 컬럼명 확인 후 수정 (이미지 기준: "소재지" 또는 "주소")
address_col = "주소"  # 또는 "주소"

# 위도/경도 컬럼 추가
latitudes = []
longitudes = []

for addr in police_location_df[address_col]:
    lat, lon = get_coordinates(addr)
    latitudes.append(lat)
    longitudes.append(lon)
    time.sleep(0.3)  # API 호출 제한 방지 (초당 최대 10회)

police_location_df["위도"] = latitudes
police_location_df["경도"] = longitudes

# 결과 확인
police_location_df.head()

,구 분,기관명,주소,위도,경도
0,동탄구,동탄2지구대,경기도 화성시 동탄영천로 70,37.208987,127.103885
1,동탄구,동탄3지구대,경기도 화성시 왕배산길 6,37.181215,127.122567
2,동탄구,동탄지구대,경기도 화성시 노작로 226-1,37.207494,127.077415
3,동탄구,반월파출소,경기도 화성시 영통로 27번길 25-19,37.230828,127.062445
4,동탄구,안용파출소,경기도 화성시 용주로 32번길 4,37.205915,127.013322


In [182]:
police_location_df

,구 분,기관명,주소,위도,경도
0,동탄구,동탄2지구대,경기도 화성시 동탄영천로 70,37.208987,127.103885
1,동탄구,동탄3지구대,경기도 화성시 왕배산길 6,37.181215,127.122567
2,동탄구,동탄지구대,경기도 화성시 노작로 226-1,37.207494,127.077415
3,동탄구,반월파출소,경기도 화성시 영통로 27번길 25-19,37.230828,127.062445
4,동탄구,안용파출소,경기도 화성시 용주로 32번길 4,37.205915,127.013322
5,동탄구,정남파출소,경기도 화성시 정남면 만년로 582,37.172975,126.983975
6,동탄구,태안지구대,경기도 화성시 떡전골로 112-2,37.208353,127.033246
7,분당구,구미파출소,경기도 성남시 분당구 구미로 107,37.339464,127.120738
8,분당구,금곡지구대,경기도 성남시 분당구 성남대로 171번길 12,37.351617,127.107284
9,분당구,동판교파출소,경기도 성남시 분당구 동판교로 266번길 24,37.406604,127.115742


In [ ]:
# 저장 
police_location_df.to_csv('../../Preprocessing/외부안전위험요소/경기도성남시_지구대_파출소_위치정보.csv', index=False)

In [183]:
# NaN인 행만 추출
df_nan = police_location_df[police_location_df['위도'].isna()].copy()

# 주소 앞부분만 잘라서 재시도 (상세주소 제거)
def shorten_address(address):
    """
    '경기도 성남시 수정구 시민로 150 3층' 
    → '경기도 성남시 수정구 시민로 150'
    번지/층수 등 상세주소 제거
    """
    return ' '.join(str(address).split()[:4])  # 앞 4단어만 사용

for idx, row in df_nan.iterrows():
    short_addr = shorten_address(row['주소'])
    lat, lon = get_coordinates(short_addr)
    police_location_df.at[idx, '위도'] = lat
    police_location_df.at[idx, '경도'] = lon
    time.sleep(0.1)

# 재시도 후 여전히 NaN인 행 확인
still_nan = police_location_df[police_location_df['위도'].isna()]
print(f"재시도 후 남은 NaN: {len(still_nan)}개")
print(still_nan['주소'].values)

재시도 후 남은 NaN: 0개
[]


In [186]:
police_location_df.to_csv('../../Preprocessing/외부안전위험요소/성남화성시_지구대_파출서_위치정보.csv', index=False)

In [148]:
# 문제 주소로 응답 직접 출력해서 확인

url = "https://dapi.kakao.com/v2/local/search/address.json"
headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
params = {"query": "경기도 성남시 중원구 산성대로408번길 26-1"}

response = requests.get(url, headers=headers, params=params)
print(response.status_code)  # 200이면 정상, 401이면 키 문제
print(response.json())       # 실제 응답 내용 확인

403
{'errorType': 'NotAuthorizedError', 'message': 'App(어린이보호구역 ) disabled OPEN_MAP_AND_LOCAL service.'}


## 6. 경기도 성남시_도로상 조명시설 지오태깅 데이터_20221216
https://www.data.go.kr/data/15110582/fileData.do

- 수집일자: 2022-12-16 ~ 2025-11-13

In [ ]:
lamp_df = pd.read_csv('./raw_data/경기도 성남시_도로상 조명시설 지오태깅 데이터_20221216.csv', encoding='cp949')
lamp_df.head()

,순번(NO),구분(LAMP),일련번호(SER),위도(LATITUDE),경도(LONGITUDE)
0,1,2,고기로-1,37.352327,127.082932
1,2,2,고기로-2,37.352327,127.082932
2,3,2,고기로-3,37.352327,127.082932
3,4,2,구미로-1,37.343812,127.124693
4,5,2,구미로-2,37.343812,127.124693


In [200]:
lamp_df.rename(columns={'위도(LATITUDE)': '위도', '경도(LONGITUDE)': '경도'}, inplace=True)

In [201]:
lamp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33568 entries, 0 to 33567
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   순번(NO)     33568 non-null  int64  
 1   구분(LAMP)   33568 non-null  int64  
 2   일련번호(SER)  33568 non-null  object 
 3   위도         33568 non-null  float64
 4   경도         33568 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.3+ MB


In [220]:
from sklearn.neighbors import BallTree
import numpy as np

def find_nearest(point_df, safezone_df, k=1):
    # BallTree 생성 (haversine은 라디안 단위 필요)
    tree = BallTree(
        np.radians(safezone_df[['위도', '경도']].values),
        metric='haversine'
    )
    
    # 쿼리: 각 포인트마다 가장 가까운 1개 찾기
    _, nearest_idx = tree.query(
        np.radians(point_df[['위도', '경도']].values),
        k=k)
    
    return safezone_df.iloc[nearest_idx.flatten()][['시설종류', '대상시설명']].values

In [224]:
result = find_nearest(lamp_df, safezone_df, k=1)
lamp_df[['시설종류', '대상시설명']] = result  # 시설종류와 대상시설명 컬럼 추가
lamp_df.drop(columns = '순번(NO)', inplace=True)
lamp_df.head()

,구분(LAMP),일련번호(SER),위도,경도,시설종류,대상시설명
0,2,고기로-1,37.352327,127.082932,초등학교,늘푸른초등학교
1,2,고기로-2,37.352327,127.082932,초등학교,늘푸른초등학교
2,2,고기로-3,37.352327,127.082932,초등학교,늘푸른초등학교
3,2,구미로-1,37.343812,127.124693,유치원,성모 유치원
4,2,구미로-2,37.343812,127.124693,유치원,성모 유치원


In [231]:
lamp_count = pd.DataFrame(lamp_df.groupby('대상시설명').size(), columns=['조명개수'])
lamp_count.reset_index(inplace=True)
lamp_count

,대상시설명,조명개수
0,갈보리어린이집,64
1,건영장안유치원,139
2,검단초등학교,228
3,고등나래 유치원,414
4,구미동어린이집,565
...,...,...
134,해나유치원,491
135,혜성유치원,195
136,휴맥스어린이집,158
137,희망대초등학교,166


In [237]:
# 구역별 조명 개수 
lamp_count.to_csv('../../Preprocessing/외부안전위험요소/어린이보호구역별_조명개수.csv', index=False)

## 7.경기도 성남시_인구및세대_현황_20260131
https://www.data.go.kr/data/15007386/fileData.do

- 수집일자: ~2026-02-03

In [ ]:
population_df = pd.read_csv('./raw_data/경기도 성남시_인구및세대_현황_20260131.csv')
population_df.head()

,구별,동,인구수_계,인구수_남,인구수_여,19세 이상_계,19세 이상_남,19세 이상_여,65세 이상_계,65세 이상_남자,65세 이상_여자,세대수,재외국민,데이터기준일자
0,수정구,신흥1동,11999,6416,5583,11277,6041,5236,3142,1377,1765,7417,19,2026-01-31
1,수정구,신흥2동,31620,15355,16265,26664,12853,13811,5248,2426,2822,12302,49,2026-01-31
2,수정구,신흥3동,10632,5718,4914,9957,5373,4584,2616,1227,1389,6436,9,2026-01-31
3,수정구,태평1동,13508,7011,6497,12791,6659,6132,3599,1591,2008,8214,22,2026-01-31
4,수정구,태평2동,13600,6853,6747,12562,6350,6212,3388,1453,1935,7525,17,2026-01-31


In [ ]:
population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   구별         50 non-null     object
 1   동          50 non-null     object
 2   인구수_계      50 non-null     int64 
 3   인구수_남      50 non-null     int64 
 4   인구수_여      50 non-null     int64 
 5   19세 이상_계   50 non-null     int64 
 6   19세 이상_남   50 non-null     int64 
 7   19세 이상_여   50 non-null     int64 
 8   65세 이상_계   50 non-null     int64 
 9   65세 이상_남자  50 non-null     int64 
 10  65세 이상_여자  50 non-null     int64 
 11  세대수        50 non-null     int64 
 12  재외국민       50 non-null     int64 
 13  데이터기준일자    50 non-null     object
dtypes: int64(11), object(3)
memory usage: 5.6+ KB


=> 아동 수가 없지만 일단 패스하겠다..

## 8. 202512_202512_연령별인구현황_연간
https://jumin.mois.go.kr/ageStatMonth.do#none

- 수집 기간: 25년 (25년 1년간 )

In [ ]:
child_population_df = pd.read_csv('./raw_data/202512_202512_연령별인구현황_연간.csv', encoding='cp949')
child_population_df.head()

,행정구역,2025년_계_총인구수,2025년_계_연령구간인구수,2025년_계_0~4세,2025년_계_5~9세,2025년_계_10~14세,2025년_계_15~19세,2025년_남_총인구수,2025년_남_연령구간인구수,2025년_남_0~4세,2025년_남_5~9세,2025년_남_10~14세,2025년_남_15~19세,2025년_여_총인구수,2025년_여_연령구간인구수,2025년_여_0~4세,2025년_여_5~9세,2025년_여_10~14세,2025년_여_15~19세
0,경기도 성남시 (4113000000),"905,794","135,396","23,841","30,320","40,148","41,087","447,380","69,062","12,224","15,421","20,581","20,836","458,414","66,334","11,617","14,899","19,567","20,251"
1,경기도 성남시 수정구 (4113100000),"232,389","28,654","5,925","6,772","7,725","8,232","117,449","14,523","3,039","3,434","3,886","4,164","114,940","14,131","2,886","3,338","3,839","4,068"
2,경기도 성남시 수정구 신흥1동(4113151000),"12,046",810,101,139,248,322,"6,439",416,53,72,128,163,"5,607",394,48,67,120,159
3,경기도 성남시 수정구 신흥2동(4113152000),"31,635","5,231","1,621","1,410","1,078","1,122","15,368","2,624",810,727,543,544,"16,267","2,607",811,683,535,578
4,경기도 성남시 수정구 신흥3동(4113153000),"10,670",748,122,143,198,285,"5,735",381,68,68,95,150,"4,935",367,54,75,103,135


In [ ]:
child_population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   행정구역             54 non-null     object
 1   2025년_계_총인구수     54 non-null     object
 2   2025년_계_연령구간인구수  54 non-null     object
 3   2025년_계_0~4세     54 non-null     object
 4   2025년_계_5~9세     54 non-null     object
 5   2025년_계_10~14세   54 non-null     object
 6   2025년_계_15~19세   54 non-null     object
 7   2025년_남_총인구수     54 non-null     object
 8   2025년_남_연령구간인구수  54 non-null     object
 9   2025년_남_0~4세     54 non-null     object
 10  2025년_남_5~9세     54 non-null     object
 11  2025년_남_10~14세   54 non-null     object
 12  2025년_남_15~19세   54 non-null     object
 13  2025년_여_총인구수     54 non-null     object
 14  2025년_여_연령구간인구수  54 non-null     object
 15  2025년_여_0~4세     54 non-null     object
 16  2025년_여_5~9세     54 non-null     object
 17  2025년_여_10~14세   54 non-null     obje